# SD Card Decoding

## Protocol Outline


The protocol employs an 8-LED NeoPixel strip to encode hexadecimal data. The LEDs are numbered 0 through 7. Each LED can take Red, Green, and Blue values, with each value ranging from 0 to 255. The actual displayed color of the LED is determined by the combination of these three values.

1. **Packet Start**:
   - A new packet begins with LED 0 displaying solid White (255,255,255) for 2 seconds. LEDs 1-7 are off at this time.

Here's an ASCII representation of the NeoPixel strip at the start of a packet:

```
LED: 0 1 2 3 4 5 6 7
     ↓
     Solid White (Packet Start)
```

2. **Channel Indication and Data Encoding**:
   - After the 2-second delay, LED 0 is set to pure Red, Green, or Blue (255 intensity on the chosen color and 0 on the other two). This indicates the data channel for the current packet.
   - Simultaneously, hex data is encoded into the color intensities on LEDs 1-7. The chosen channel carries the actual data, and the other two channels are set to random intensities.
   - Each LED from 1-7 represents one byte of hex data, encoding a total of 7 bytes per packet on the chosen channel.

After the delay, LED 0 is set to pure Red, Green, or Blue to indicate the chosen channel, and data is simultaneously encoded into the color intensities on LEDs 1-7. In this example, we choose the Green channel, so LED 0 will be pure green (0,255,0) and the Green value will carry the actual data on LEDs 1-7:

```
LED: 0           1           2           3           4           5           6           7
R    (0)         RND         RND         RND         RND         RND         RND         RND
G    (255)       DATA        DATA        DATA        DATA        DATA        DATA        DATA
B    (0)         RND         RND         RND         RND         RND         RND         RND
```

Remember, the color displayed by each LED is the combination of the Red, Green, and Blue values. Therefore, LEDs won't show pure colors (except for LED 0 at the channel indication step).

3. **Additional Notes**:
   - Each packet will be broadcast for 3 seconds
   - After each broadcast, LEDs will be cleared (turned off) until the next packet is broadcasted.
   - After all packets have been broadcasted, LEDs will turn off. 

## Challenge

### Arduino Src Code


```c
#include <Adafruit_NeoPixel.h>

#define PIN            6
#define NUMPIXELS      8
#define DELAY_MS       2000
#define TIME_BTWN_PACKET 3000

Adafruit_NeoPixel pixels(NUMPIXELS, PIN, NEO_GRB + NEO_KHZ800);
char channels[] = {'R', 'G', 'B'};
uint8_t data[] = "ACSI{n0t_v3ryyy_s3cur33_m3ssag1ng_5y5t3m_fdb8ca26}";
bool toSend = true;

void clear() {
  for (int i = 0; i < NUMPIXELS; i++) {
    pixels.setPixelColor(i, pixels.Color(0, 0, 0));
  }
  pixels.show();
}

void sendData(char channel, uint8_t* data, int len) {
  //indicate start of packet and Set LED 0 to White
  clear();
  pixels.setPixelColor(0, pixels.Color(255, 255, 255));
  pixels.show();
  delay(DELAY_MS);
  
  // Set the LED 0 to the corresponding channel
  if (channel == 'R') {
    pixels.setPixelColor(0, pixels.Color(255, 0, 0));
  } else if (channel == 'G') {
    pixels.setPixelColor(0, pixels.Color(0, 255, 0));
  } else if (channel == 'B') {
    pixels.setPixelColor(0, pixels.Color(0, 0, 255));
  }
    
  // Send data
  for (int i = 0; i < 7; i++) {
    uint8_t r = channel == 'R' ? (i < len ? data[i] : 0) : random(255);
    uint8_t g = channel == 'G' ? (i < len ? data[i] : 0) : random(255);
    uint8_t b = channel == 'B' ? (i < len ? data[i] : 0) : random(255);
    
    pixels.setPixelColor(i + 1, pixels.Color(r, g, b));
  }
  
  //display
  pixels.show();
}

void setup() {
  // Init Serial
  Serial.begin(115200);

  pixels.begin();
  // Clear strip
  clear();
}

void loop() {
  // wait for serial input before executing following code. once executed, never execute again
  while (!Serial.available());
  String input = Serial.readString();
  if (input != "start") {
    Serial.println("Invalid input");
    return;
  } else if (input == "start" && toSend) {
      Serial.println("Starting");
      // Send data in chunks
      for (int i = 0; i < sizeof(data); i += 7) {
        char channel = channels[random(3)];
        sendData(channel, data + i, min(sizeof(data) - i, 7));
        delay(TIME_BTWN_PACKET);
      }
      clear();
      toSend = false;
      Serial.println("Done");
  }
}
```

### Solution

In [1]:
import csv

def decode_led(file_name):
    with open(file_name, 'r') as f:
        reader = csv.reader(f)
        next(reader)  # Skip the header

        data_packets = []
        current_packet = []
        start_packet = False
        data_channel_found = False
        data_channel_index = 0

        for row in reader:
            #strip '0x' and convert to int
            index = int(row[4][2:], 16)
            pixel = [int(row[5][2:], 16) , int(row[6][2:], 16), int(row[7][2:], 16)] #R,G,B

            # Check if we've found packet start (LED 0 flashes white)
            if index == 0 and pixel == [255, 255, 255]:
                start_packet = True
                continue

            # get data channel after packet start (either red, green or blue)
            if start_packet:
                if index == 0:
                    start_packet = False
                    # data_channel_index is the index of the element in pixel array with value 255
                    data_channel_index = pixel.index(255)
                    data_channel_found = True
                    continue

            # Start collecting data after data channel is found
            if data_channel_found:
                current_packet.append(pixel[data_channel_index])

            # Stop collecting data after LED clears (i.e. pixel = [0, 0, 0]) or when we reach 7 bytes
            if data_channel_found and (len(current_packet) == 7 or pixel == [0, 0, 0]):
                data_packets.append(current_packet)
                current_packet = []
                data_channel_found = False

    # Convert blocks to ASCII and print
    for packet in data_packets:
        print(''.join(chr(p) for p in packet), end='')

In [2]:
decode_led('led.csv')

ACSI{n0t_v3ryyy_s3cur33_m3ssag1ng_5y5t3m_fdb8ca26}      